In [38]:
# utils
import pandas as pd
import os
import numpy as np
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import ast

# evaluation 
from evaluate import load

In [39]:
# open files in generations directory
def open_files(directory):
    files = os.listdir(directory)
    files.sort()
    return files

In [40]:
datasets = {}

files = open_files('generations')
for file in files:
    dataset = pd.read_csv('generations/' + file)
    datasets[file] = dataset
files

['llama2_7b_it_2epoch_decoding.csv',
 'llama2_7b_it_sa_1shot_2epoch_decoding.csv',
 'llama2_7b_it_sa_2epoch_decoding.csv',
 'llamantino2_7b_it_2epoch_decoding.csv',
 'llamantino2_7b_it_sa_2epoch_decoding.csv']

In [41]:
datasets['llama2_7b_it_2epoch_decoding.csv']

,data_unit,predictions,actuals
0,11th_Mississippi_Infantry_Monument category Co...,Il Monumento dell'11° Fanteria del Mississippi...,"Un monumento all'11° Fanteria del Mississippi,..."
1,Bananaman broadcastedBy BBC BBC city Broadcast...,"Bananaman è stato trasmesso dalla BBC, che ha ...",Bill Oddie è protagonista di un programma dell...
2,Karlsruhe postalCode 76131–76229,Il codice postale di Karlsruhe è 76131-76229.,I codici postali di Karlsruhe sono 76131-76229.
3,Turkish_people religion Irreligion,La religione dei turchi è l'irreligione.,Alcuni turchi sono irreligiosi.
4,ENAIRE city Madrid Adolfo_Suárez_Madrid–Baraja...,L'aeroporto Adolfo Suarez Madrid-Barajas si tr...,L'aeroporto Adolfo Suarez Madrid-Barajas è ges...
...,...,...,...
995,"ALCO_RS-3 buildDate ""May 1950 - August 1956"" A...","L'ALCO RS-3, costruito tra il maggio 1950 e l'...",L'ALCO RS-3 prodotto tra il maggio 1950 e l'ag...
996,Super_Capers budget 2000000.0 Super_Capers dir...,Super Capers è stato diretto da Ray Griggs e i...,Super Capers è un film di 98 minuti distribuit...
997,It's_Great_to_Be_Young_(1956_film) editing Max...,Max Benedict ha curato la regia del film del 1...,It's Great to Be Young è stato curato da Max B...
998,Lady_Anne_Monson deathDate 1776-02-18 Lady_Ann...,Lady Anne Monson era una cittadina del Regno d...,"Lady Anne Monson, sposata con George Monson, e..."


In [42]:
# foreach file
for file in files:
    dataset = datasets[file]

    # save in an array the index of the rows with the data unit with less then 14 spaces
    indexes = []
    for index, row in dataset.iterrows():
        if row['data_unit'].count(' ') < 12:
            indexes.append(index)
    break

print(len(indexes))

559


In [43]:
# foreach file
for file in files:
    dataset = datasets[file]

    # remove the rows in the indexes array
    dataset = dataset.drop(indexes)

    # get 50 random samples with a seed of 42
    samples = dataset.sample(200, random_state=42)

    # for each sample count the number of space in the data unit
    print(samples['data_unit'].apply(lambda x: x.count(' ')))

    # create an xlsx file with only data units and predictions cols

    # aggiungi anche la colonna actual se presente
    if 'actuals' in samples.columns:
        samples = samples[['data_unit', 'predictions', 'actuals']]
    else:
        samples = samples[['data_unit', 'predictions']]
    

    samples.to_excel('human_eval_sets/' + file + '.xlsx', index=False)

    # save also in csv format
    samples.to_csv('human_eval_sets/' + file + '.csv', index=False)


166    12
998    12
334    12
602    12
876    15
       ..
981    15
439    14
455    12
421    12
200    33
Name: data_unit, Length: 200, dtype: int64
166    16
998    16
334    16
602    16
876    20
       ..
981    20
439    18
455    16
421    16
200    39
Name: data_unit, Length: 200, dtype: int64
166    16
998    16
334    16
602    16
876    20
       ..
981    20
439    18
455    16
421    16
200    39
Name: data_unit, Length: 200, dtype: int64
166    12
998    12
334    12
602    12
876    15
       ..
981    15
439    14
455    12
421    12
200    33
Name: data_unit, Length: 200, dtype: int64
166    16
998    16
334    16
602    16
876    20
       ..
981    20
439    18
455    16
421    16
200    39
Name: data_unit, Length: 200, dtype: int64
